## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-19-19-02-43 +0000,world,Canadian woman dies after falling ill at Belgi...,https://www.cbc.ca/news/world/tomorrowland-dea...
1,2025-07-19-18-54-55 +0000,world,Israeli troops fatally shoot 32 Palestinians t...,https://www.cbc.ca/news/world/aid-site-shootin...
2,2025-07-19-17-13-12 +0000,world,"Tour boat in Vietnam capsizes during storm, ki...",https://www.cbc.ca/news/world/vietnam-tour-boa...
3,2025-07-19-14-38-28 +0000,world,"Congo, M23 rebels pledge to reach peace deal n...",https://www.cbc.ca/news/world/congo-m23-rebels...
4,2025-07-19-13-47-44 +0000,world,Vehicle strikes people waiting to enter L.A. n...,https://www.cbc.ca/news/world/los-angeles-car-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2421/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,reach,2
0,canadian,1
2,dies,1
1,woman,1
4,ill,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-07-19-08-01-13 +0000,world,CEO resigns after being caught cosying up to c...,https://www.cbc.ca/news/coldplay-viral-video-s...,12
1,2025-07-19-18-54-55 +0000,world,Israeli troops fatally shoot 32 Palestinians t...,https://www.cbc.ca/news/world/aid-site-shootin...,11
5,2025-07-19-12-35-24 +0000,world,Clashes escalate in Syria's Druze region despi...,https://www.cbc.ca/news/world/syria-ceasefire-...,10
0,2025-07-19-19-02-43 +0000,world,Canadian woman dies after falling ill at Belgi...,https://www.cbc.ca/news/world/tomorrowland-dea...,9
3,2025-07-19-14-38-28 +0000,world,"Congo, M23 rebels pledge to reach peace deal n...",https://www.cbc.ca/news/world/congo-m23-rebels...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,12,2025-07-19-08-01-13 +0000,world,CEO resigns after being caught cosying up to c...,https://www.cbc.ca/news/coldplay-viral-video-s...
1,11,2025-07-19-18-54-55 +0000,world,Israeli troops fatally shoot 32 Palestinians t...,https://www.cbc.ca/news/world/aid-site-shootin...
5,10,2025-07-19-12-35-24 +0000,world,Clashes escalate in Syria's Druze region despi...,https://www.cbc.ca/news/world/syria-ceasefire-...
0,9,2025-07-19-19-02-43 +0000,world,Canadian woman dies after falling ill at Belgi...,https://www.cbc.ca/news/world/tomorrowland-dea...
2,7,2025-07-19-17-13-12 +0000,world,"Tour boat in Vietnam capsizes during storm, ki...",https://www.cbc.ca/news/world/vietnam-tour-boa...
3,6,2025-07-19-14-38-28 +0000,world,"Congo, M23 rebels pledge to reach peace deal n...",https://www.cbc.ca/news/world/congo-m23-rebels...
4,6,2025-07-19-13-47-44 +0000,world,Vehicle strikes people waiting to enter L.A. n...,https://www.cbc.ca/news/world/los-angeles-car-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
